In [ ]:
from google.colab import drive
drive.mount('/content/drive')

APP = "/content/drive/MyDrive/heartriskx/app.py"
import os
if not os.path.exists(APP):
    raise FileNotFoundError(f"{APP} not found.")
print("✅ Found:", APP)


Mounted at /content/drive
✅ Found: /content/drive/MyDrive/heartriskx/app.py


In [ ]:
%%bash
pip -q install streamlit

# Install cloudflared if missing
if [ ! -x /usr/local/bin/cloudflared ]; then
  echo "Installing cloudflared..."
  curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 \
       -o /usr/local/bin/cloudflared
  chmod +x /usr/local/bin/cloudflared
else
  echo "cloudflared already present:"
  /usr/local/bin/cloudflared --version
fi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.9 MB/s eta 0:00:00
Installing cloudflared...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  27.5M      0  0:00:01  0:00:01 --:--:-- 40.3M


In [ ]:
import subprocess, time, re, os, sys

APP = "/content/drive/MyDrive/heartriskx/app.py"
PORT = 8501

# Start Streamlit in background
st_log = "/content/streamlit.log"
open(st_log, "w").close()
st = subprocess.Popen(
    ["streamlit", "run", APP, "--server.port", str(PORT), "--server.address", "0.0.0.0"],
    stdout=open(st_log,"a"), stderr=subprocess.STDOUT, text=True
)
print("Starting Streamlit… (logs at", st_log, ")")
time.sleep(2)

# Start cloudflared and capture the public URL
cf = subprocess.Popen(
    ["/usr/local/bin/cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

public_url = None
deadline = time.time() + 60  # wait up to 60s
while time.time() < deadline:
    line = cf.stdout.readline()
    if not line:
        time.sleep(0.2); continue
    m = re.search(r"https://[-a-z0-9]+\.trycloudflare\.com", line)
    if m:
        public_url = m.group(0)
        break

if not public_url:
    # show last logs to help debug
    cf.kill()
    print("❌ Could not obtain public URL. Last lines from cloudflared:")
    try:
        cf_out = cf.stdout.read()
        print(cf_out[-1000:] if cf_out else "(no output)")
    except Exception:
        pass
    raise SystemExit

print("\n🌐 PUBLIC URL:", public_url)
print("Tip: first load may show 502 for a few seconds—refresh once.")


Starting Streamlit… (logs at /content/streamlit.log )

🌐 PUBLIC URL: https://declaration-daisy-liberty-lawn.trycloudflare.com
Tip: first load may show 502 for a few seconds—refresh once.
